**En el presente archivo se llevará a cabo del desarrollo del código necesario para crear el proyecto**

# **1-** ETL

Comenzamos con el proceso de ETL (extract, transform and load), siendo este el primer paso hacia la generación de conocimiento a partir de datos. En esta etapa se busca conocer la información con la que contamos, las diferentes estrucuras y formatos que mantienen los datos, así como las relaciones entre los datasets, con el objetivo de lograr la integración de los mismos para su posterior utilización de forma eficiente y efectiva. 

---

Importamos las librerías a utilizar

In [2]:
import pandas as pd
import json
import gzip
import ast


Comenzamos con la extracción de los datos. Para esto se disponibilizaron 3 datasets en formato comprimido .gz:
- steam_games.json.gz
- user_reviews.json.gz
- user_items.json.gz

## 1.1 df_games

In [3]:
# Leemos el archivo 'steam_games.json.gz' y lo guardamos, utilizando la funcion read_json de pandas
df_games = pd.read_json('steam_games.json.gz', compression= 'gzip',lines=True)
print('tamanio_games:', df_games.shape)
df_games.head(3)

tamanio_games: (120445, 19)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,76561197970982479,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,js41637,7.656120e+16,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,None,evcentric,7.656120e+16,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0


Se observa que algunos campos son los mismos contiene user_items por lo cual se decide eliminarlos

In [4]:
df_games.drop(columns=['user_id','steam_id','items','items_count'], axis=1, inplace=True)
df_games.shape

(120445, 15)

Analizamos los valores nulos

In [10]:
df_games.isnull().sum()

publisher          96362
genres             91593
app_name           88312
title              90360
url                88310
release_date       90377
tags               88473
reviews_url        88312
discount_price    120220
specs              88980
price              89687
early_access       88310
id                 88312
metascore         117768
developer          91609
dtype: int64

Se procede a eliminar filas vacías

In [5]:
df_games.dropna(how='all',inplace=True)
df_games.reset_index(drop=True,inplace=True)
print(df_games.shape)
df_games.head(3)

(32135, 15)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,0.0,761140.0,None,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,None,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,NaN,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,None,Poolians.com


In [8]:
# Corroboramos los cambios
df_games.isnull().sum()

publisher          8052
genres             3283
app_name              2
title              2050
url                   0
release_date       2067
tags                163
reviews_url           2
discount_price    31910
specs               670
price              1377
early_access          0
id                    2
metascore         29458
developer          3299
dtype: int64

In [10]:
# exploramos un poco el comportamiento y la estructura del df resultante utilizando un filtro en el campo {id}
filtro_loc = df_games.loc[df_games['id'] == 10]
filtro_loc

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
32106,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",9.99,0.0,10.0,88,Valve


In [6]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   reviews_url     32133 non-null  object 
 8   discount_price  225 non-null    float64
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  float64
 12  id              32133 non-null  float64
 13  metascore       2677 non-null   object 
 14  developer       28836 non-null  object 
dtypes: float64(3), object(12)
memory usage: 3.7+ MB


In [ ]:
# una vez listo este dataset lo exportamos para tener esta version 
df_games.to_csv('df_games.csv', index=False)

## 1.2 df_reviews

Leemos el archivo 'user_reviews.json.gz' corriendo una función ya que traia errores de sintaxis y no se podía leer simplemente con pd.read_json, en este caso utilizamos la libreria -gzip- para poder abrir un archivo comprimido sin necesidad de descomprimirlo previamente y el modulo -ast- junto con la funcion literal_eval para sortear los errores del archivo.

In [5]:
data = []

for i in gzip.open('user_reviews.json.gz'):
     data.append(ast.literal_eval(i.decode('utf-8')))                                   

df_reviews = pd.DataFrame(data)
print('Tamanio_reviews:',df_reviews.shape)
df_reviews.head(5)

Tamanio_reviews: (25799, 3)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Antes de explorar los datos anidados analizamos los valores nulos

In [15]:
df_reviews.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

Revisamos los datos anidados recorriendo uno de sus campos iterando el df con iterrows

In [14]:
for index, row in df_reviews.iterrows():
    lista = row['reviews']
    user_id = row['user_id']
    for i in lista:
        a = (user_id, i['posted'])
        print(a)


('76561197970982479', 'Posted November 5, 2011.')
('76561197970982479', 'Posted July 15, 2011.')
('76561197970982479', 'Posted April 21, 2011.')
('js41637', 'Posted June 24, 2014.')
('js41637', 'Posted September 8, 2013.')
('js41637', 'Posted November 29, 2013.')
('evcentric', 'Posted February 3.')
('evcentric', 'Posted December 4, 2015.')
('evcentric', 'Posted November 3, 2014.')
('evcentric', 'Posted October 15, 2014.')
('evcentric', 'Posted October 15, 2014.')
('evcentric', 'Posted October 15, 2014.')
('doctr', 'Posted October 14, 2013.')
('doctr', 'Posted July 28, 2012.')
('doctr', 'Posted June 2, 2012.')
('doctr', 'Posted June 29, 2014.')
('doctr', 'Posted November 22, 2012.')
('doctr', 'Posted February 23, 2012.')
('maplemage', 'Posted April 15, 2014.')
('maplemage', 'Posted December 23, 2013.')
('maplemage', 'Posted March 14, 2014.')
('maplemage', 'Posted July 11, 2013.')
('Wackky', 'Posted May 5, 2014.')
('Wackky', 'Posted December 24, 2012.')
('Wackky', 'Posted October 21, 201

Observamos que gran parte de los datos anidados serán consumidos por lo cual procedemos a desanidar y acomodar el df para lo que viene

In [6]:
# Utilizamos explode y json_normalize para desanidar la columna "reviews"
exp = df_reviews.explode('reviews')
df_desanidado = pd.json_normalize(exp['reviews'])

# Por otra parte reindexamos la columna "user_id" con los registros resultantes 
df1 = exp['user_id'].reset_index()

# Concatenamos la columna "user_id" al df desanidado
df_desanidado['user_id'] = df1['user_id']

# Ahora df_desanidado contiene las claves de la columna anidada
# y la columnas "user_id" correspondiente. Por ultimo asignamos un nombre representativo.

df_reviews = df_desanidado
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [14]:
# comenzamos a explorar los datos y su estructura
print(df_reviews.columns)
df_reviews.shape

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')


(59333, 8)

In [16]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  object
 6   review       59305 non-null  object
 7   user_id      59333 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


Revisamos valores nulos

In [17]:
df_reviews.isnull().sum()

funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
user_id         0
dtype: int64

In [17]:
filas_con_nulos = df_reviews[df_reviews['item_id'].isnull()]
filas_con_nulos     

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gdxsd
177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198094224872
2559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198021575394
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cmuir37
13767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jaysteeny
15493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ML8989
19184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198079215291
20223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198079342142
25056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198061996985
26257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561198108286351


In [7]:
# Vemos que son todos id sin ningun dato asi que los eliminamos
df_reviews.dropna(inplace=True)

In [8]:
df_reviews.isnull().sum()

funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
user_id        0
dtype: int64

Revisamos duplicados

In [9]:
df_reviews.duplicated().sum()

874

In [10]:
filas_duplicadas = df_reviews[df_reviews.duplicated()]
filas_duplicadas  

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
1114,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep,bokkkbokkk
2894,,"Posted January 10, 2014.",,218620,1 of 3 people (33%) found this review helpful,True,"Good graphics, fun heists! A bit laggy",ImSeriouss
2895,,"Posted January 10, 2014.",,105600,0 of 2 people (0%) found this review helpful,True,So fun! DEFINITELY NOT RIP OFF OF MINECRAFT! e...,ImSeriouss
2896,,"Posted December 17, 2014.",,570,No ratings yet,True,bobo pinoy,ImSeriouss
2897,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...,ImSeriouss
...,...,...,...,...,...,...,...,...
44456,,Posted July 3.,,422400,No ratings yet,True,Muy entretenido y una coleccion de armas prome...,76561198092022514
44457,,Posted June 1.,,218620,No ratings yet,True,"Tiene una jugabilidad y tematica muy buena :D,...",76561198092022514
44458,,"Posted August 17, 2014.",,261820,No ratings yet,True,"Buen juego, no importa el desarrrollo que tien...",76561198092022514
44459,,"Posted February 17, 2014.",,224260,No ratings yet,True,exelente aporte :D¡¡¡ es una buen mod basado e...,76561198092022514


In [11]:
# Eliminamos filas duplicadas
df_reviews.drop_duplicates(inplace=True)

In [12]:
df_reviews.duplicated().sum()

0

In [23]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58431 entries, 0 to 59332
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        58431 non-null  object
 1   posted       58431 non-null  object
 2   last_edited  58431 non-null  object
 3   item_id      58431 non-null  object
 4   helpful      58431 non-null  object
 5   recommend    58431 non-null  object
 6   review       58431 non-null  object
 7   user_id      58431 non-null  object
dtypes: object(8)
memory usage: 4.0+ MB


## 1.3 df_items

In [3]:
# leemos el archivo 'users_items.json.gz' igual al de reviews
data = []

for i in gzip.open('users_items.json.gz'):
     data.append(ast.literal_eval(i.decode('utf-8')))                                   

df_items = pd.DataFrame(data)

df_items.head(5)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [ ]:
df_items.shape

Al igual que con el dataset de reviews, se procede a desanidar los datos de la columna 'items' la cual cuenta con información de utilidad para el objetivo del proyecto. 

In [4]:
# Utilizamos explode y json_normalize para desanidar la columna "items"
exp = df_items.explode('items')
df_desanidado = pd.json_normalize(exp['items'])

# Por otra parte eliminamos la columna anidada y reindexamos  
exp1 = exp.drop(columns='items').reset_index(drop=True)

# Concatenamos 
df_items = pd.concat([df_desanidado, exp1],axis=1)

# Ahora df_items contiene todas las columnas a un mismo nivel
print(df_items.shape)
df_items.head()

### Análisis de valores nulos

In [28]:
exp1.isnull().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
dtype: int64

In [29]:
df_desanidado.isnull().sum()

item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [27]:
df_desanidado.dropna(how='all',inplace=True)

In [28]:
df_desanidado.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
dtype: int64

In [32]:
df_items.isnull().sum()

item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
user_id                 0
items_count             0
steam_id                0
user_url                0
dtype: int64

Se observa que los nulos en el df resultante son todos nulos presentes en la columna anidada por lo cual se procede a eliminarlos

In [16]:
df_items.dropna(subset=['item_id','item_name','playtime_forever','playtime_2weeks'], inplace=True)

In [30]:
df_items.isnull().sum()

item_id             0
item_name           0
playtime_forever    0
playtime_2weeks     0
user_id             0
items_count         0
steam_id            0
user_url            0
dtype: int64

In [35]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 5170013
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
 4   user_id           object 
 5   items_count       int64  
 6   steam_id          object 
 7   user_url          object 
dtypes: float64(2), int64(1), object(5)
memory usage: 353.8+ MB


De acuerdo al análisis exploratorio realizado hasta aquí, se considera que una buena decisión en torno a la eficiencia y mejor practicidad de trabajo posterior es unificar los dataframe df_games y df_items en uno solo llamado df_items_game

In [34]:
df_items.item_id.isnull().sum()

0

In [36]:
# analizamos los nulos de la columna id de df_games que es la que nos interesa para hacer el merge con item_id 
# de df_items
df_items['id'].isnull().sum()

2

In [35]:
filas_con_nulos = df_games[df_games['id'].isnull()]
filas_con_nulos                              

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
74,None,None,None,None,http://store.steampowered.com/,None,None,None,14.99,None,19.99,0.0,NaN,None,None
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,91,"Rocksteady Studios,Feral Interactive (Mac)"


De la revision anterior podemos observar que la url contiene el id del item, de esta forma podemos completar el valor faltante en la columna id con '200260' para el indice 30961, el otro nulo lo eliminamos porque no hay informacion suficiente para obtener el id

In [36]:
# revisamos antes del cambio y vemos que hay solo un registro para ese id
filtro_loc = df_games.loc[df_games['id'] == 200260]
filtro_loc

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,91,"Rocksteady Studios,Feral Interactive (Mac)"


In [17]:
# actualizamos el valor
df_games.at[30961, 'id'] = 200260

In [39]:
# revisamos luego del cambio
filtro_loc = df_games.loc[df_games['id'] == 200260]
filtro_loc

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,91,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,91,"Rocksteady Studios,Feral Interactive (Mac)"


In [40]:
filas_con_nulos = df_games[df_games['id'].isnull()]
filas_con_nulos      

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
74,None,None,None,None,http://store.steampowered.com/,None,None,None,14.99,None,19.99,0.0,NaN,None,None


In [18]:
# vemos que ahora solo queda un valor nulo al cual eliminamos
df_games.dropna(subset=['id'],inplace=True)

In [43]:
df_games['id'].isnull().sum()

0

Procedemos a realizar el merge

In [19]:
# hacemos una verificación previa de estos datos
print(df_items['item_id'][0])
df_games['id'][0]

10


761140.0

In [20]:
# Pasamos los valores de item_id e id a 'int' 
df_items['item_id'] = df_items.item_id.astype(int)
df_games['id'] = df_games.id.astype(int)

In [47]:
item_id = df_items.loc[df_items['item_id']== 10]
item_id.head(3)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
277,10,Counter-Strike,0.0,0.0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
1302,10,Counter-Strike,0.0,0.0,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch


In [48]:
id_games = df_games.loc[df_games['id'] == 10]
id_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer
32106,Valve,[Action],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",9.99,0.0,10,88,Valve


In [21]:
#corregimos el nombre de las columnas para que se llamen iguales "item_id"
df_games.rename(columns={'id':'item_id'}, inplace=True)

In [22]:
# realizamos el merge 
df_items_games = df_items.merge(df_games, on="item_id", how="left")

In [23]:
df_items_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5161872 entries, 0 to 5161871
Data columns (total 22 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
 4   user_id           object 
 5   items_count       int64  
 6   steam_id          object 
 7   user_url          object 
 8   publisher         object 
 9   genres            object 
 10  app_name          object 
 11  title             object 
 12  url               object 
 13  release_date      object 
 14  tags              object 
 15  reviews_url       object 
 16  discount_price    float64
 17  specs             object 
 18  price             object 
 19  early_access      float64
 20  metascore         object 
 21  developer         object 
dtypes: float64(4), int32(1), int64(1), object(16)
memory usage: 846.7+ MB


In [39]:
df_items_games['item_id'].sort_values()

0              10
2455218        10
2456667        10
2456731        10
4863134        10
            ...  
3792369    529670
104961     529670
3305558    529820
557304     530720
586824     530720
Name: item_id, Length: 5161872, dtype: int32

In [40]:
df_games['item_id'].sort_values()

32106         10
32103         20
32114         30
32108         40
32104         50
          ...   
30593    2028055
30689    2028056
30322    2028062
30524    2028103
1425     2028850
Name: item_id, Length: 32134, dtype: int32

---

# **2-** Desarrollo de funciones

Para trabajar mejor cada una de las funciones se realizará previamente una exploración específica de los datos enfocada en aquellos que requiere cada función para alcanzar el resultado buscado. De esta forma mejoramos el desempeño y la eficiencia de procesamiento a la vez que conseguimos una mejor organización de los datos. 

---

## **2.1- def userdata**( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

Datos requeridos: df_items_game.user_id, df_items_game.price, df_items_game.items_count y df_reviews.user_id, df_reviews.recommend 

In [95]:
df_userdata_1 = df_items_games[['user_id','price',]]
df_userdata_2 = df_reviews[['user_id','recommend']]

In [89]:
df_userdata_1.head(3)

,user_id,price
0,76561197970982479,9.99
1,76561197970982479,4.99
2,76561197970982479,4.99


In [107]:
df_userdata_2.head()

,user_id,recommend
0,76561197970982479,True
1,76561197970982479,True
2,76561197970982479,True
3,js41637,True
4,js41637,True


In [40]:
# Revisamos valores diferentes en 'price'
df_userdata_1['price'].unique()

array([9.99, 4.99, 19.99, 6.99, 7.99, 2.99, 14.99, 'Free', 54.99, 29.99,
       39.99, 24.99, 1.99, 'Free to Play', 'Free To Play', 'Third-party',
       11.99, 0.49, 0.98, 0.99, 59.99, 3.99, 2.49, 74.76, 5.99, 32.99,
       12.99, 49.99, 13.98, 8.99, 13.99, 'Install Now', 34.99,
       'Play WARMACHINE: Tactics Demo', 12.89, 20.0, 14.95, 61.99, 15.99,
       18.99, 17.99, 'Free Movie', 160.91, 59.95, 44.99, 3.49, 1.87,
       'Free to Use', 23.99, 21.99, 15.0, 'Play for Free!', 'Free Mod',
       189.96, 139.92, 23.96, 26.99, 16.99, 'Free HITMAN™ Holiday Pack',
       7.49, 10.93, 19.98, 87.94, 'Play the Demo', 69.99, 16.06, 13.37,
       10.99, 79.99, 1.0, 'Free to Try', 0.5, 49.0, 771.71, 20.99, 99.99,
       4.49, 9.95, 124.99, 10.0, 0.89, 1.29, 18.9, 4.0, 3.0, 2.0, 31.99,
       119.99, 9.69, 299.99, 29.96, 9.98, 1.5, 8.98, 12.0, 9.0, 5.65,
       74.99, 2.89, 'Starting at $449.00'], dtype=object)

In [97]:
# como en price hay valores no numericos corregimos
df_userdata_1['price'] = pd.to_numeric(df_userdata_1['price'], errors='coerce')

C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_42140\26727772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_userdata_1['price'] = pd.to_numeric(df_userdata_1['price'], errors='coerce')


In [42]:
df_userdata_1['price'].unique()

array([9.9900e+00, 4.9900e+00, 1.9990e+01, 6.9900e+00, 7.9900e+00,
       2.9900e+00, 1.4990e+01,        nan, 5.4990e+01, 2.9990e+01,
       3.9990e+01, 2.4990e+01, 1.9900e+00, 1.1990e+01, 4.9000e-01,
       9.8000e-01, 9.9000e-01, 5.9990e+01, 3.9900e+00, 2.4900e+00,
       7.4760e+01, 5.9900e+00, 3.2990e+01, 1.2990e+01, 4.9990e+01,
       1.3980e+01, 8.9900e+00, 1.3990e+01, 3.4990e+01, 1.2890e+01,
       2.0000e+01, 1.4950e+01, 6.1990e+01, 1.5990e+01, 1.8990e+01,
       1.7990e+01, 1.6091e+02, 5.9950e+01, 4.4990e+01, 3.4900e+00,
       1.8700e+00, 2.3990e+01, 2.1990e+01, 1.5000e+01, 1.8996e+02,
       1.3992e+02, 2.3960e+01, 2.6990e+01, 1.6990e+01, 7.4900e+00,
       1.0930e+01, 1.9980e+01, 8.7940e+01, 6.9990e+01, 1.6060e+01,
       1.3370e+01, 1.0990e+01, 7.9990e+01, 1.0000e+00, 5.0000e-01,
       4.9000e+01, 7.7171e+02, 2.0990e+01, 9.9990e+01, 4.4900e+00,
       9.9500e+00, 1.2499e+02, 1.0000e+01, 8.9000e-01, 1.2900e+00,
       1.8900e+01, 4.0000e+00, 3.0000e+00, 2.0000e+00, 3.1990e

In [106]:
def userdata(User_id: str) -> dict:
    '''Dado un id de usuario igresado, la función retorna:
    - La cantidad de juegos que adquirió el usuario
    - El dinero gastado por el usuario
    - El % de juegos que recomienda de todos los que adquirió
    '''
    if type(User_id) is not str:
        raise TypeError("El parametro debe ser string")
                 
    # Filtra el DataFrame para obtener solo las filas correspondientes al User_id proporcionado
    data1 = df_userdata_1[df_userdata_1['user_id'] == User_id]
    data2 = df_userdata_2[df_userdata_2['user_id'] == User_id]
    
    # Obtiene la Cantidad de items comprados por el usuario extrayendo el valor del campo 'items_count'
    num_items = len(data1) - 1 
    
    # Calcula la cantidad de dinero gastado por el usuario (suma de la columna 'price')
    Dinero_gastado = data1['price'].sum()
    
    # Calcula el porcentaje de recomendación del usuario
    recommend_percentage = data2.recommend.sum()/len(data2)*100
    

    
    # Crea un diccionario con los resultados
    user_info = {
        'User_id': User_id,
        'Cantidad_de_items': num_items,
        'Dinero_gastado': Dinero_gastado,
        'Porcentaje_recomendacion': recommend_percentage
    }
    
    return user_info

In [108]:
userdata('js41637')

{'User_id': 'js41637',
 'Cantidad_de_items': 888,
 'Dinero_gastado': 8573.06,
 'Porcentaje_recomendacion': 100.0}

---

## **2.2- def countreviews**( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

Datos requeridos: df_reviews.user_id, df_reviews.posted y df_reviews.recommend

In [84]:
df_reviews.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479


Observamos que las fechas en el campo 'posted' no estan en el formato solicitado como argumento, por lo cual procedemos a cambiarlo. Para esto creamos una función

In [102]:
import re
from datetime import datetime

# Función para convertir la fecha en formato 'Posted November 5, 2011.' a '2011-11-05'
def convert_date(date_str):
    # Define un diccionario para mapear nombres de meses a números de mes
    month_mapping = {
        'January': '01',
        'February': '02',
        'March': '03',
        'April': '04',
        'May': '05',
        'June': '06',
        'July': '07',
        'August': '08',
        'September': '09',
        'October': '10',
        'November': '11',
        'December': '12'
    }

    # Utiliza una expresión regular para extraer los componentes de la fecha
    date_match = re.match(r'Posted (\w+) (\d+), (\d+)', date_str)
    if date_match:
        month_name, day, year = date_match.groups()
        # Obtiene el número de mes del diccionario de mapeo
        month = month_mapping.get(month_name)
        # Formatea la fecha en 'YYYY-MM-DD'
        formatted_date = f'{year}-{month}-{day.zfill(2)}'
        return formatted_date

    # Si no se puede extraer la fecha, devuelve None
    return None

# Ejemplo de uso
date_str = 'Posted November 5, 2011.'
formatted_date = convert_date(date_str)
print(formatted_date)  # Salida: '2011-11-05'


2011-11-05


In [103]:
# Creamos una nueva columna y le aplicamos la funcion creada 
df_reviews['formatted_date'] = df_reviews['posted'].apply(convert_date)

In [104]:
# Creamos el df especifico para esta funcion
df_countreviews = df_reviews[['user_id','recommend','formatted_date']]
df_countreviews

,user_id,recommend,formatted_date
0,76561197970982479,True,2011-11-05
1,76561197970982479,True,2011-07-15
2,76561197970982479,True,2011-04-21
3,js41637,True,2014-06-24
4,js41637,True,2013-09-08
...,...,...,...
59328,76561198312638244,True,None
59329,76561198312638244,True,None
59330,LydiaMorley,True,None
59331,LydiaMorley,True,None


In [197]:
# Conviertimos la columna 'fecha' al tipo DateTime si aún no lo está
df_countreviews['formatted_date'] = pd.to_datetime(df_countreviews['formatted_date'])

C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_10192\72366107.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countreviews['formatted_date'] = pd.to_datetime(df_countreviews['formatted_date'])


In [89]:
def countreviews(start_date: str, end_date: str):
    """
    La presente función calcula la cantidad de usuarios que realizaron reviews entre las fechas ingresadas y, 
    el porcentaje de recomendación de los mismos en base a sus reviews.

    Args:
        start_time (str): fecha de inicio en formato 'YYYY-MM-DD'
        end_time (str): fecha de fin en formato 'YYYY-MM-DD'

    """
    # Convierte las fechas de inicio y fin en objetos datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filtra las reseñas que están dentro del rango de fechas dado
    filtered_reviews = df_countreviews[(df_countreviews['formatted_date'] >= start_date) & (df_countreviews['formatted_date'] <= end_date)]

    # Obtiene la cantidad de usuarios únicos que realizaron reseñas en ese rango
    unique_users = filtered_reviews['user_id'].nunique()

    # Calcula el porcentaje de recomendación
    recommendation_percentage = round((filtered_reviews['recommend'].sum() / filtered_reviews.shape[0]) * 100, 2) 

    result = {
        'Cantidad de usuarios que realizaron reviews': unique_users,
        'Porcentaje de recomendaciones': recommendation_percentage
    }

    return result

In [90]:
countreviews('2011-11-05','2013-09-08')

{'Cantidad de usuarios que realizaron reviews': 2167,
 'Porcentaje de recomendaciones': 98.16}

---

## **2.3- def genre**( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

Datos requeridos: df_items_games.genrer y df_items_games.playtime_forever

En este caso se debe hacer un tratamiento posterior al df creado ya que la columna 'genres' contiene datos anidados que debemos desanidar para luego agrupar la suma de horas por genero. Por último ordenamos de mayor a menor segun la cantidad de horas y llegamos a un df con el ranking listo para utilizar

In [113]:
# Genero un dataframe a partir de las columnas que necesito de df_items_games
df_genres = df_items_games[['genres','playtime_forever']].explode('genres')
# Agrupo por genero sumando todos los valores de playtime_forever
df_genres = df_genres.groupby('genres')['playtime_forever'].sum().reset_index()
# Ordenamos el dataframe en orden descendente en base a 'playtime_forever'
df_genres = df_genres.sort_values(by='playtime_forever', ascending=False).reset_index(drop=True)
# Convierto la columna 'genres' a un formato consistente (primera letra en mayúscula, resto en minúsculas).
df_genres['genres'] = df_genres['genres'].str.lower().str.capitalize()
# Agrego la columna de ranking (opcional)
df_genres['ranking'] = range(1, len(df_genres) + 1)

df_genres

,genres,playtime_forever,ranking
0,Action,3.118833e+09,1
1,Indie,1.494622e+09,2
2,Rpg,1.041023e+09,3
3,Adventure,9.152658e+08,4
4,Simulation,8.676463e+08,5
5,Strategy,6.593638e+08,6
6,Free to play,6.107529e+08,7
7,Massively multiplayer,4.465941e+08,8
8,Casual,2.522329e+08,9
9,Early access,1.587013e+08,10


In [114]:
print(df_genres[df_genres['genres']=='Action'].ranking.values)

[1]


In [115]:
def genre( genero : str ): 
    '''
    La función recibe el nombre de un género y retorna el puesto que ocupa el mismo en el ranking de los 
    género más jugados por los usuarios

    Args:
        genero (str): Género a buscar dentro de la base de datos.

    '''
    # validacion de argumento ingresado
    if type(genero) is not str:
        raise TypeError("El argumento debe ser de tipo string")
    # Convierte el género especificado a un formato consistente (primera letra en mayúscula, resto en minúsculas).
    genero = genero.lower().capitalize()
    # Filtra el DataFrame en función del género especificado.
    df_filtrado = df_genres[df_genres['genres'] == genero]
    
    # Verifica si el género especificado se encuentra en el DataFrame.
    if df_filtrado.empty:
        return print("El género especificado no se encuentra en la base de datos.") # El género no se encuentra en el DataFrame.
    
    # Obtiene el índice de la fila correspondiente al género especificado.
    puesto = df_filtrado.index[0].item() + 1  # Sumar 1 para obtener un ranking basado en 1 en lugar de 0.
    
    result = {
        'Género': genero,
        'Ranking N°': puesto
    }

    return result
    #return print(f"El género {genero} ocupa el puesto #{puesto} en el ranking.")

Evaluamos la función con diferentes ejemplos

In [116]:
genre('aventura')

El género especificado no se encuentra en la base de datos.


In [117]:
genre('action')

El género Action ocupa el puesto #1 en el ranking.


## **2.4- def userforgenre**( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

Datos requeridos: df_items_games[['user_id','genres','playtime_forever','user_url']]

In [133]:
df_items_games.shape

(5161872, 22)

Nuevamente creamos el df requerido y desanidamos la columna 'genres'. En este caso la agrupación debe hacerse dentro de la función ya que depende del argumento ingresado

In [148]:
df_top5 = df_items_games[['user_id','genres','playtime_forever','user_url']].explode('genres')
df_top5

,user_id,genres,playtime_forever,user_url
0,76561197970982479,Action,6.0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,Action,0.0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,Action,7.0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,Action,0.0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,Action,0.0,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...
5161869,76561198329548331,Free to Play,3.0,http://steamcommunity.com/profiles/76561198329...
5161870,76561198329548331,Casual,4.0,http://steamcommunity.com/profiles/76561198329...
5161870,76561198329548331,Free to Play,4.0,http://steamcommunity.com/profiles/76561198329...
5161870,76561198329548331,Indie,4.0,http://steamcommunity.com/profiles/76561198329...


In [149]:
# Se agrupan los datos por usuario y url sumando las horas jugadas por genero.
df_top5 = df_top5.groupby(['user_id', 'user_url', 'genres'])['playtime_forever'].sum().reset_index()

Para optimizar el tamanio del df resultante analizamos la forma de recortarlo.
Una de ellas es eliminando los registros que tienen 0 horas jugadas ya que no aportan informacion relevante

In [157]:
df_top5[df_top5.playtime_forever == 0]

,user_id,user_url,genres,playtime_forever
16,--ace--,http://steamcommunity.com/id/--ace--,Massively Multiplayer,0.0
20,--ace--,http://steamcommunity.com/id/--ace--,Sports,0.0
37,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,Massively Multiplayer,0.0
41,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,Sports,0.0
168,-Mad-,http://steamcommunity.com/id/-Mad-,Early Access,0.0
...,...,...,...,...
671304,zzonci,http://steamcommunity.com/id/zzonci,Action,0.0
671307,zzonci,http://steamcommunity.com/id/zzonci,Indie,0.0
671323,zzydrax,http://steamcommunity.com/id/zzydrax,Casual,0.0
671325,zzydrax,http://steamcommunity.com/id/zzydrax,Free to Play,0.0


In [160]:
# Nos quedamos solo con valores mayores a 0 
df_top5 = df_top5[df_top5['playtime_forever'] != 0]

In [161]:
df_top5[df_top5.playtime_forever == 0]

,user_id,user_url,genres,playtime_forever


In [162]:
df_top5[df_top5.genres == 'Action'].sort_values(by='playtime_forever', ascending=False).reset_index(drop=True)

,user_id,user_url,genres,playtime_forever
0,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,Action,1700533.0
1,shinomegami,http://steamcommunity.com/id/shinomegami,Action,1580843.0
2,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,Action,1457865.0
3,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,Action,1078197.0
4,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,Action,1061193.0
...,...,...,...,...
66892,76561198080847604,http://steamcommunity.com/profiles/76561198080...,Action,1.0
66893,zadow,http://steamcommunity.com/id/zadow,Action,1.0
66894,76561198091015208,http://steamcommunity.com/profiles/76561198091...,Action,1.0
66895,76561198095484410,http://steamcommunity.com/profiles/76561198095...,Action,1.0


In [163]:
df_top5.shape

(631131, 4)

In [165]:

def userforgenre(genero:str):
    '''
    La presente función recibe como argumento el nombre de un género (str) y devuelve el 
    Top 5 de usuarios con más horas de juego en dado género, con su URL y user_id.
    '''
    #validacion de argumento
    if not isinstance(genero, str):
        raise TypeError("El argumento 'género' debe ser una cadena (str)")
    
    # Filtramos el DataFrame para incluir solo las filas relacionadas con el género especificado.
    df_filtrado = df_top5[df_top5['genres'].str.contains(genero, case=False, na=False)]

    if df_filtrado.empty:
        return print("No se encontraron registros para el género", genero)
    
    # Se agrupan los datos por usuario y url sumando las horas jugadas.
    df_agrupado = df_filtrado.groupby(['user_id','user_url'])['playtime_forever'].sum().reset_index()

    # Ordena los usuarios en función de las horas jugadas en orden descendente.
    df_ordenado = df_agrupado.sort_values(by='playtime_forever', ascending=False).reset_index(drop=True)

    # Selecciona los primeros 5 usuarios del ranking.
    top_5_usuarios = df_ordenado.head(5).reset_index(drop=True)

    # convertimos el resultado en un diccionario
    resultado = top_5_usuarios.to_dict(orient='list')

    return resultado


In [167]:
userforgenre('Action')

{'user_id': ['Sp3ctre',
  'shinomegami',
  'REBAS_AS_F-T',
  'Terminally-Chill',
  'DownSyndromeKid'],
 'user_url': ['http://steamcommunity.com/id/Sp3ctre',
  'http://steamcommunity.com/id/shinomegami',
  'http://steamcommunity.com/id/REBAS_AS_F-T',
  'http://steamcommunity.com/id/Terminally-Chill',
  'http://steamcommunity.com/id/DownSyndromeKid'],
 'playtime_forever': [1700533.0, 1580843.0, 1457865.0, 1078197.0, 1061193.0]}

In [121]:
userforgenre('aventura')

No se encontraron registros para el género aventura


In [122]:
userforgenre('aCtIoN')

{'user_id': ['Sp3ctre',
  'shinomegami',
  'REBAS_AS_F-T',
  'Terminally-Chill',
  'DownSyndromeKid'],
 'user_url': ['http://steamcommunity.com/id/Sp3ctre',
  'http://steamcommunity.com/id/shinomegami',
  'http://steamcommunity.com/id/REBAS_AS_F-T',
  'http://steamcommunity.com/id/Terminally-Chill',
  'http://steamcommunity.com/id/DownSyndromeKid'],
 'playtime_forever': [1700533.0, 1580843.0, 1457865.0, 1078197.0, 1061193.0]}

***

## **2.5- def developer**( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

Datos requeridos: df_items_games[['developer','price','item_id','release_date']]

In [111]:
df_developer = df_items_games[['developer','price','item_id','release_date']]
df_developer

,developer,price,item_id,release_date
0,Valve,9.99,10,2000-11-01
1,Valve,4.99,20,1999-04-01
2,Valve,4.99,30,2003-05-01
3,Valve,4.99,40,2001-06-01
4,Gearbox Software,4.99,50,1999-11-01
...,...,...,...,...
5161867,Reperio Studios,Free,346330,2016-07-20
5161868,None,Free,373330,None
5161869,CoaguCo Industries,Free to Play,388490,2015-09-01
5161870,Tamationgames,Free To Play,521570,2016-08-24


In [112]:
# Analizamos que datos tenemos en relacion a los precios
df_developer.price.unique()

array([9.99, 4.99, 19.99, nan, 6.99, None, 7.99, 2.99, 14.99, 'Free',
       54.99, 29.99, 39.99, 24.99, 1.99, 'Free to Play', 'Free To Play',
       'Third-party', 11.99, 0.49, 0.98, 0.99, 59.99, 3.99, 2.49, 74.76,
       5.99, 32.99, 12.99, 49.99, 13.98, 8.99, 13.99, 'Install Now',
       34.99, 'Play WARMACHINE: Tactics Demo', 12.89, 20.0, 14.95, 61.99,
       15.99, 18.99, 17.99, 'Free Movie', 160.91, 59.95, 44.99, 3.49,
       1.87, 'Free to Use', 23.99, 21.99, 15.0, 'Play for Free!',
       'Free Mod', 189.96, 139.92, 23.96, 26.99, 16.99,
       'Free HITMAN™ Holiday Pack', 7.49, 10.93, 19.98, 87.94,
       'Play the Demo', 69.99, 16.06, 13.37, 10.99, 79.99, 1.0,
       'Free to Try', 0.5, 49.0, 771.71, 20.99, 99.99, 4.49, 9.95, 124.99,
       10.0, 0.89, 1.29, 18.9, 4.0, 3.0, 2.0, 31.99, 119.99, 9.69, 299.99,
       29.96, 9.98, 1.5, 8.98, 12.0, 9.0, 5.65, 74.99, 2.89,
       'Starting at $449.00'], dtype=object)

Como no necesitamos valores sino diferenciar los juegos gratis de los pagos procedemos a llevar la columna 'price' a esta forma.

In [113]:
# llevamos todo a str y minusculas
df_developer['price'] = df_developer['price'].str.lower()
# como no necesitamos los precios, pasamos los valores que tenian valor a 'pago' 
df_developer['price'].fillna('paid',inplace=True)
# Eliminar caracteres especiales
df_developer['price'] = df_developer['price'].str.replace(r'[^\w\s]', '', regex=True)
# unificamos los valores de price por free y paid
df_developer['price'].replace(['free', 'free hitman holiday pack','free to play', 'free to try','free movie',
                               'free to use','play for free','free mod',], 'free', inplace=True)
df_developer['price'].replace(['thirdparty', 'install now','play warmachine tactics demo', 'play the demo',
                               'starting at 44900'], 'paid', inplace=True)
# Convertir la columna 'release_date' a formato de fecha
df_developer['release_date'] = pd.to_datetime(df_developer['release_date'], errors='coerce')
# Extraer el año de lanzamiento y crear una nueva columna 'release_year'
df_developer['release_year'] = df_developer['release_date'].dt.year
# Borramos la columna 'release_date'
del df_developer["release_date"]
#normalizamos mayusculas y minusculas en 'developer'
df_developer['developer'] = df_developer['developer'].str.lower().str.capitalize()

C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_42140\944798998.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer['price'] = df_developer['price'].str.lower()
C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_42140\944798998.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer['price'].fillna('paid',inplace=True)
C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_42140\944798998.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [114]:
print(df_developer.price.unique())
df_developer.head()

['paid' 'free']


,developer,price,item_id,release_year
0,Valve,paid,10,2000.0
1,Valve,paid,20,1999.0
2,Valve,paid,30,2003.0
3,Valve,paid,40,2001.0
4,Gearbox software,paid,50,1999.0


Analizamos los valores nulos del df resultante

In [115]:
df_developer.isnull().sum()

developer       965111
price                0
item_id              0
release_year    961623
dtype: int64

In [116]:
# los registros que no tienen anio directamente los eliminamos considerando que no podemos obtener 
# ese dato con la informacion disponible
df_developer.dropna(subset='release_year', inplace=True)

C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_42140\420104150.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer.dropna(subset='release_year', inplace=True)


In [117]:
df_developer.isnull().sum()

developer       34374
price               0
item_id             0
release_year        0
dtype: int64

In [124]:
# vemos si podemos obtener info de los developers que nos faltan
filas_con_nulos = df_developer[df_developer['developer'].isnull()]
lista = filas_con_nulos.item_id.unique()
lista

array([239350, 248490, 248710, 252370, 253650, 247120, 259780, 347350,
       354500, 355150, 245550, 247910, 315260, 374570,  33420, 351570,
       361700, 242590, 242940, 242980, 242550, 252030, 393360, 413850,
       254100,  11390, 411180, 253920, 242960, 214190, 246300, 349210,
       406210, 227020, 396160, 243040, 362280, 230860, 253940, 449680,
       425680,  33730,  12690, 247350, 253310, 367930, 367950, 357470,
       259700,  11370, 306260, 335900, 252770, 248730,   9730, 248970,
       245010, 243060, 245730, 240660, 259490, 260510, 371270, 380890,
       382640, 393970, 398970, 412860, 413600, 452220, 519530, 357380,
       492510, 357410, 387560, 387620, 249950, 387660, 463750, 474230,
       489910, 508280, 365170, 237370, 288290, 382630, 441540, 371240,
       394920, 394930, 394940, 357460, 359410, 371230, 382620, 382650,
       382660, 382680, 382690, 382700, 382710, 382720, 382740, 382750,
       382760, 382770, 382780, 382790, 387460, 387470, 387480, 387490,
      

In [122]:
dev = df_games.developer[df_games.item_id==10].values
dev[0]

'Valve'

In [125]:
# corroboramos si algun id sin valor en developer tiene el valor en otro registro
for i in lista:
    dev = df_games.developer[df_games.item_id==i].values
    if dev[0] is not None:
        print(i, dev)

In [126]:
# No hay info entonces eliminamos nulos de developer
df_developer = df_developer.dropna(subset=['developer']) 

In [127]:
df_developer.isnull().sum()

developer       0
price           0
item_id         0
release_year    0
dtype: int64

In [128]:
# pasamos el valor de los anios a int
df_developer.release_year = df_developer.release_year.astype(int)
df_developer.release_year.min()

C:\Users\LENOVO IDEAPAD\AppData\Local\Temp\ipykernel_42140\1969225452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_developer.release_year = df_developer.release_year.astype(int)


1983

In [129]:
def developer(developer:str):
    '''
    'developer' calcula la cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.
    
    Args:
        developer (str): nombre de la empresa desarrolladora
    Return:
        diccionario con la cantidad total de items y el % de items gratis en cada año para esa empresa
    '''
    # Validacion de argumento
    if not isinstance(developer, str):
        raise TypeError("El argumento debe ser una cadena (str)")
    # Convierte el texto ingresado a un formato consistente (primera letra en mayúscula, resto en minúsculas).
    developer = developer.lower().capitalize()
    # Filtra el DataFrame para obtener solo los juegos del desarrollador especificado
    df_filter = df_developer[df_developer['developer'] == developer]

    # Validacion de dato
    if df_filter.empty:
        return {"mensaje": f"No se encontraron registros para la empresa {developer}"}

    # Agrupa por año y calcula la cantidad de juegos (item_id) y el porcentaje de contenido gratuito
    grouped = df_filter.groupby('release_year').agg(
        games_total=('item_id', 'count'),
        games_Free=('price', lambda x: (x == 'free').mean() * 100)
    ).reset_index()

    resultado = grouped.to_dict(orient='list')
    return resultado

Evaluamos la función

In [130]:
developer('Gearbox softWARE')

{'release_year': [1999, 2001, 2005, 2008, 2009, 2011, 2016],
 'games_total': [6289, 6206, 508, 304, 9121, 2171, 1188],
 'games_Free': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0]}

In [131]:
developer('one BOX')

{'mensaje': 'No se encontraron registros para la empresa One box'}

In [132]:
developer(123)

TypeError: El argumento debe ser una cadena (str)

---

## **2.6- def sentiment_analysis**( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Datos requeridos: df_reviews[['review','formatted_date']]

In [141]:
df_sentiment = df_reviews[['review','formatted_date']]

In [144]:
df_sentiment

,review,formatted_date
0,Simple yet with great replayability. In my opi...,2011-11-05
1,It's unique and worth a playthrough.,2011-07-15
2,Great atmosphere. The gunplay can be a bit chu...,2011-04-21
3,I know what you think when you see this title ...,2014-06-24
4,For a simple (it's actually not all that simpl...,2013-09-08
...,...,...
59328,a must have classic from steam definitely wort...,None
59329,this game is a perfect remake of the original ...,None
59330,had so much fun plaing this and collecting res...,None
59331,:D,None


In [ ]:
# Pasamos los datos a tipo date
df_sentiment['formatted_date'] = pd.to_datetime(df_sentiment['formatted_date'])

La consigna pide que si hay valores faltantes de reviews lo consideremos como neutral, para eso le pasamos un review neutral a los valores faltantes

In [ ]:
df_sentiment['review'].fillna('My opinion about this game is neutral', inplace=True)

In [ ]:
# A los registros sin fecha de publicacion los eliminamos ya que no tenemos forma de conocer esos datos
df_sentiment.dropna(inplace=True)

In [149]:
# Analizamos si hay duplicados
df_sentiment.duplicated().sum()

100

In [ ]:
# Se decide eliminar los duplicados, si bien no se puede asegurar 100% que realmente sean registros totalmente duplicados
# para los fines de esta funcion se conisera adecuado quitarlos
df_sentiment.drop_duplicates(inplace=True) 

In [152]:
df_sentiment.duplicated().sum()

0

In [156]:
df_sentiment.shape

(48398, 2)

Desarrollamos una función para realizar el análisis de sentimiento a las reseñas. Para esto utilizamos NLTK (Natural Language Toolkit) una librería open source de Python para procesamiento de lenguaje natural.

In [153]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Inicializar el analizador de sentimiento VADER
analyzer = SentimentIntensityAnalyzer()

# Función para asignar un valor numérico al sentimiento
def asignar_sentimiento(texto):
    sentiment = analyzer.polarity_scores(texto)
    compound_score = sentiment['compound']
    if compound_score >= 0.05:
        return 2  # Positivo
    elif compound_score <= -0.05:
        return 0  # Malo
    else:
        return 1  # Neutral

In [154]:
# Creamos un DataFrame de ejemplo para testear la funcion 
data = {'id_usuario': [1, 2, 3],
        'resena': ["I love this games",
                   "My opinion about this game is neutral",
                   "I didnt like nothing, its so bad"]}
df = pd.DataFrame(data)
# Aplicamos la función de asignación de sentimiento a la columna 'resena'
df['sentimiento'] = df['resena'].apply(asignar_sentimiento)
print(df)

   id_usuario                                 resena  sentimiento
0           1                      I love this games            2
1           2  My opinion about this game is neutral            1
2           3       I didnt like nothing, its so bad            0


In [ ]:
# Aplicamos la funcion a nuestro df
df_sentiment['sentiment'] = df_sentiment['review'].apply(asignar_sentimiento)
df_sentiment.head(10)

In [224]:
print('fecha mas antigua', df_sentiment.formatted_date.min())
print('fecha mas reciente', df_sentiment.formatted_date.max())

fecha mas antigua 2010-10-16 00:00:00
fecha mas reciente 2015-12-31 00:00:00


In [185]:
for i in (0,1,2):
    print('valor',i,df_sentiment.review[df_sentiment.sentiment == i].values[85])

valor 0 "Stay in the light, don't be seen, don't listen to the voices, stay in the LIGHT, don't look at the cor- no nononono! Don't be seen, don't make any noise, don't look at the..."
valor 1 AMAZEING GAME :Sans:
valor 2 Best game I have ever played and a thousand times even better when you download mods


Creamos la función

In [219]:
def sentiments_analysis(year:int) -> dict:
    '''
    Esta función recibe un año como argumento y cuenta la cantidad de reseñas negativas (0),
    neutrales (1) y positivas (2) en ese año.

    Args:
        year (int): El año en formato numérico YYYY.

    Returns:
        dict: Un diccionario con la cantidad de cada tipo de sentimiento.
    '''
    # Validación de argumento 
    list_anios = df_sentiment.formatted_date.dt.year.to_list()
    if year not in list_anios:
        return print('No hay registros para el año ingresado')
    # Filtrar el DataFrame por el año especificado
    df_filtered = df_sentiment[df_sentiment['formatted_date'].dt.year == year]

    # Contar la cantidad de cada tipo de sentimiento
    sentiment_counts = df_filtered['sentiment'].value_counts().to_dict()

    # Crear un diccionario con las cantidades
    result_dict = {
        'Negative (0)': sentiment_counts.get(0, 0),
        'Neutral (1)': sentiment_counts.get(1, 0),
        'Positive (2)': sentiment_counts.get(2, 0)
    }

    return result_dict


Evaluación

In [226]:
sentiments_analysis(2013)

{'Negative (0)': 754, 'Neutral (1)': 1322, 'Positive (2)': 4612}

In [225]:
sentiments_analysis(2020)

No hay registros para el año ingresado


# **3**- Exportación de archivos 

Para mejorar el desempeño de la api que desarrollaremos luego, exportamos todos los dataframes creados para cada función a archivos parquet, de modo de optimizar la utilización de memoria.

In [227]:
df_userdata_1.to_parquet('df_userdata_1.parquet')
df_userdata_2.to_parquet('df_userdata_2.parquet')
df_countreviews.to_parquet('df_countreviews.parquet')
df_genres.to_parquet('df_genres.parquet')
df_top5.to_parquet('df_top5.parquet')
df_developer.to_parquet('df_developer.parquet')
df_sentiment.to_parquet('df_sentiment.parquet')